In [1]:
import tools210922
import yfinance as yf
import pandas as pd

In [2]:
con = tools210922.connect_postgres("azure_eltr")

In [3]:
t = yf.Ticker("AMZN")
tools210922.insert_symbol(con, t)

Eintrag existiert schon


In [4]:
t.quarterly_earnings

,Revenue,Earnings
Quarter,,
3Q2021,110812000000,3156000000
4Q2021,137412000000,14323000000
1Q2022,116444000000,-3844000000
2Q2022,121234000000,-2028000000


In [ ]:
# Tabelle aus Ticker holen
df = t.quarterly_earnings
# Umformen, damit passend zu SQL-Tabelle
symbol = t.info["symbol"]
df["symbol"] = symbol
df["quarter"] = df.index
df["quarter"] = df["quarter"].str[2:] + "Q" + df["quarter"].str[0]
df = df.rename(columns={"Revenue":"revenue", "Earnings":"earnings"})
df
# in Datenbank hochladen
df.to_sql("earnings_quarter", con, 
    schema="finance", if_exists="append", index=False)

In [6]:
def insert_earnings_quarter(con, ticker):
    # Tabelle aus Ticker holen
    df = ticker.quarterly_earnings
    # Umformen, damit passend zu SQL-Tabelle
    symbol = ticker.info["symbol"]
    df["symbol"] = symbol
    df["quarter"] = df.index
    df["quarter"] = df["quarter"].str[2:] + "Q" + df["quarter"].str[0]
    df = df.rename(columns={"Revenue":"revenue", "Earnings":"earnings"})
    # in Datenbank hochladen
    df.to_sql("earnings_quarter", con, 
        schema="finance", if_exists="append", index=False)

In [ ]:
pd.read_sql("select * from finance.earnings_quarter",con)